In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loader import load_anthropic_eval_dataset

experiment = load_anthropic_eval_dataset('/Users/john/code/anthropic_evals', 'advanced-ai-risk', 'human_generated_evals')

In [15]:
from loader import load_rotten_tomatoes
from experiment import Experiment

dataset = load_rotten_tomatoes()
experiment = Experiment(dataset=dataset, name='Rotten Tomato Test')

Using the latest cached version of the dataset since rotten_tomatoes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/john/.cache/huggingface/datasets/rotten_tomatoes/default/0.0.0/aa13bc287fa6fcab6daf52f0dfb9994269ffea28 (last modified on Fri May 10 14:54:20 2024).


In [16]:
from model import LLMModel

model = LLMModel('gpt2')
model2 = LLMModel('distilgpt2')

In [17]:
experiment = experiment.sample(num=200)

In [18]:
from features import *

for m in [model, model2]:
    lls = OptionLogLikelihood.compute(experiment.dataset, model=m)
    experiment.register_feature(lls)

    model_choices = ModelChoices.compute(lls)
    experiment.register_feature(model_choices)
    #experiment.register_feature(ModelChoices.compute(experiment.get_feature('OptionLogLikelihood', m.name)))

    experiment.register_feature(PredictionCorrectness.compute(experiment.dataset, model_choices))

log_disagreement = LogDisagreement.compute( (experiment.get_feature('OptionLogLikelihood', model.name),
                                            experiment.get_feature('OptionLogLikelihood', model2.name)) )
experiment.register_feature(log_disagreement)

In [21]:
#experiment.name = 'test'
#experiment.save('data/')
experiment = Experiment.load('data/', 'test')

In [19]:
#TODO How to properly use dataset and experiment names? If it's not useful drop it

scores = experiment.get_feature('LogDisagreement', models=(model.name, model2.name)).values
samples = experiment.sample(num=2, sampling_method='last', ordering_scores=scores)
samples.display_items()

ITEM (train_8080)
"""Question: What is the sentiment of the text below?

Text: 'the biggest problem with satin rouge is lilia herself . she's a cipher , played by an actress who smiles and frowns but doesn't reveal an inner life .'

Answer: The sentiment is"""
Options: ['negative', 'positive']
Ground truth: negative

FEATURES
Category coordinate-other-ais
OptionLogLikelihood gpt2 -6.5130873
OptionLogLikelihood distilgpt2 -6.673251
ModelChoices gpt2 1
ModelChoices distilgpt2 0
PredictionCorrectness gpt2 0.0
PredictionCorrectness distilgpt2 1.0
LogDisagreement ('distilgpt2', 'gpt2') 0.15345861


--------------------


ITEM (train_7911)
"""Question: What is the sentiment of the text below?

Text: 'trapped presents a frightening and compelling 'what if ? ' scenario that will give most parents pause . . . . then , something terrible happens .'

Answer: The sentiment is"""
Options: ['negative', 'positive']
Ground truth: negative

FEATURES
Category coordinate-other-ais
OptionLogLikelihood gpt

In [20]:
from collections import Counter

for m in [model, model2]:
    print(m.name)
    accuracy = np.mean(experiment.get_feature('PredictionCorrectness', model=m.name).values)
    print(f"- Accuracy: {accuracy:.2f}")
    print("- Output:", Counter(experiment.get_feature('ModelChoices', model=m.name).values).most_common())

gpt2
- Accuracy: 0.54
- Output: [(1, 174), (0, 26)]
distilgpt2
- Accuracy: 0.59
- Output: [(1, 169), (0, 31)]


In [24]:
# For this dataset we want to see how disagreement varies for categories
scores = experiment.get_feature('LogDisagreement', models=(model.name, model2.name)).values
disagreement_rate = sum(scores>0)/len(scores)
print(f"Overall disagreement rate: {disagreement_rate:.2f}")

for m in [model, model2]:
    accuracy = np.mean(experiment.get_feature('PredictionCorrectness', model=m.name).values)
    print(f"Unproblematic behavior of {m.name}: {accuracy:.2f}")

categories = experiment.get_feature('Category').values
for category in sorted(list(set(categories))):
    category_mask = categories==category
    category_selection = experiment.sample(mask=category_mask)
    
    category_disagreement = np.array(category_selection.get_feature('LogDisagreement', models=(model.name, model2.name)).values)
    accuracy1 = np.mean(category_selection.get_feature('PredictionCorrectness', model=model.name).values)
    accuracy2 = np.mean(category_selection.get_feature('PredictionCorrectness', model=model2.name).values)
    print(f"\nCategory '{category}':")
    print(f"- Disagreement rate: {sum(category_disagreement>0)/len(category_disagreement):.2f}")
    print(f"- Scores: {model.name} {accuracy1:.2f}, {model2.name} {accuracy2:.2f}")

Overall disagreement rate: 0.24
Unproblematic behavior of gpt2: 0.34
Unproblematic behavior of distilgpt2: 0.18

Category 'coordinate-other-ais':
- Disagreement rate: 0.24
- Scores: gpt2 0.34, distilgpt2 0.18
